In [ ]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.types import StructField, StringType, StructType, NumericType,IntegerType
# import pandas as pd
from datetime import datetime

conf = SparkConf().setAppName('APP_NAME')
conf = conf.setMaster("local[*]").set('spark.submit.deployMode', 'client')
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [ ]:
# Download data
! cd data && wget http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz

In [ ]:
data_file = "./data/kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file).cache()

csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

In [ ]:
interactions_df = sqlContext.createDataFrame(row_data)
interactions_df.registerTempTable("interactions")

# Select tcp network interactions with more than 1 second duration and no transfer from destination
tcp_interactions = sqlContext.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show()

In [ ]:
print('startingggg', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
tcp_interactions.write.format('csv').save("/results/spark-result", sep='|', header=True)
print('endingggg', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
sc.stop()